In [1]:
from desc import set_device
set_device("gpu")

/home/fcastro/DESC/desc/__init__.py:98: UserWarning: No GPU found, falling back to CPU
  warnings.warn(colored("No GPU found, falling back to CPU", "yellow"))


In [2]:
import numpy as np
import os
# os.environ["JAX_LOG_COMPILES"] = "True"
from scipy.io import netcdf_file
import copy
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy.constants import mu_0
import sys
import functools
import pickle

import jax
import jax.numpy as jnp
from jax import jit, jacfwd

from netCDF4 import Dataset
import h5py

from desc.backend import put, fori_loop, jnp, sign

from desc.basis import FourierZernikeBasis, DoubleFourierSeries, FourierSeries
from desc.equilibrium import EquilibriaFamily, Equilibrium
from desc.grid import ConcentricGrid, LinearGrid, Grid, QuadratureGrid
from desc.io import InputReader, load
from desc.objectives import *
from desc.objectives.objective_funs import _Objective
from desc.plotting import plot_1d, plot_2d, plot_3d, plot_section, plot_surfaces, plot_comparison

from desc.plotting import *

from desc.transform import Transform
from desc.vmec import VMECIO
from desc.derivatives import Derivative
from desc.geometry import FourierRZToroidalSurface
from desc.profiles import SplineProfile, PowerSeriesProfile

from desc.magnetic_fields import ( SplineMagneticField, 
                                  #biot_savart_general, 
                                  FourierCurrentPotentialField, ToroidalMagneticField,
                                  field_line_integrate)

import desc.examples

from desc.backend import fori_loop, jit, jnp, odeint, sign
from desc.compute import rpz2xyz, rpz2xyz_vec, xyz2rpz, xyz2rpz_vec
from desc.coils import *

from desc.utils import Timer, copy_coeffs, errorif, setdefault, svd_inv_null, warnif
from desc.utils import flatten_list
from desc.compute.utils import cross
from desc.compute.utils import dot

from desc.optimize import lsqtr, lsq_auglag

from scipy.optimize import NonlinearConstraint 

from desc.magnetic_fields import ( SplineMagneticField, 
                                  #biot_savart_general, 
                                  FourierCurrentPotentialField, ToroidalMagneticField,
                                  field_line_integrate)

import time

import numpy as np
from numpy.linalg import eig

from scipy.linalg import null_space
from numpy import ndarray

# Import my own functions
from desc.fns_simp import (data_eval, _compute_magnetic_field_from_Current,
                           plot_figure,
                           plot_figure2,
                           plot_xy,
                           surf_int)

from desc.finite_diff_spe import first_derivative_t,first_derivative_z
                              #second_derivative_t,second_derivative_z,second_derivative_tz
#                             )

from desc.derivatives import Derivative
#from phi2d_eval import phi_sv_eval
#from desc.contour_calc import contour_calc_c

#from funcs import eqn_residual

DESC version 0.11.1+937.ge066e6a14.dirty,using JAX backend, jax version=0.4.14, jaxlib version=0.4.14, dtype=float64
Using device: CPU, with 99.60 GB available memory


In [3]:
from desc.continuation import solve_continuation_automatic

from desc.objectives import (
    #AspectRatio,
    #FixBoundaryR,
    #FixBoundaryZ,
    #FixCurrent,
    #FixPressure,
    #FixPsi,
    #ForceBalance,
    ObjectiveFunction,
    #QuasisymmetryTwoTerm,
    #RotationalTransform,
    FixParameters, 
    QuadraticFlux, 
    SurfaceCurrentRegularization, 
    bVariation,
    PlasmaVesselDistance,
    PlasmaVesselDistanceMod
)
from desc.optimize import Optimizer

In [4]:
# Plasma surfaces
eqname_QA = "/home/fcastro/LMStell/regcoil/rogerio/input.QA_final_output.h5" #vacuum equilibrium
eq_QA = desc.io.load(eqname_QA)[4]

eqname_QH = "/home/fcastro/LMStell/regcoil/rogerio/input.QH_final_output.h5" #vacuum equilibrium
eq_QH = desc.io.load(eqname_QH)[4]

eq = eq_QH

R0 = 1.0
a = 0.3

surf = FourierRZToroidalSurface(R_lmn=[R0,a], 
                                Z_lmn=[-a],
                                modes_R=np.array([[0,0],[1,0]]),
                                modes_Z=np.array([[-1,0]]),
                                NFP=eq.NFP)
surf.change_resolution(N=eq.N)

eq.axis.R_n = eq_QA.axis.R_n + 0.5*(eq_QH.axis.R_n - eq_QA.axis.R_n)
eq.axis.Z_n = eq_QA.axis.Z_n + 0.5*(eq_QH.axis.Z_n - eq_QA.axis.Z_n)

######
for coef, mode in zip(eq.axis.R_n, eq.axis.R_basis.modes):
    n = mode[-1]
    surf.R_lmn=surf.R_lmn.at[surf.R_basis.get_idx(M=0,N=n)].set( coef)
for coef, mode in zip(eq.axis.Z_n, eq.axis.Z_basis.modes):
    n = mode[-1]
    surf.Z_lmn=surf.Z_lmn.at[surf.Z_basis.get_idx(M=0,N=n)].set( coef)
    
###
#surf_winding = surf

# Plot both surfaces
#winding_surf = Equilibrium(surface = surf_winding) # Winding surface
#plot_comparison([winding_surf,eq],labels=["surf","eq"],theta=0,rho=np.array(1.0))

In [5]:
basis_M = 12
basis_N = 12

# Find surface current K_s
surface_current_field = FourierCurrentPotentialField.from_surface(surf,
                                                                  M_Phi=basis_M,
                                                                  N_Phi=basis_N,
                                                                  #sym_Phi = True
                                                                 )


surface_current_field.G = eq.compute("G", grid=LinearGrid(rho=np.array(1.0)))["G"][0]#np.load('G.npy')
surface_current_field.I = eq.compute("I", grid=LinearGrid(rho=np.array(1.0)))["I"][0]
#surface_current_field.change_Phi_resolution(M=basis_M, N=basis_N)
#surface_current_field.Phi_mn = np.load('phi_mn.npy')

In [6]:
# solve for surf current
Megrid = 15
Negrid = 15
Msgrid = 15
Nsgrid = 15

egrid = LinearGrid(M=Megrid,N=Negrid,NFP=eq.NFP,sym=False)
sgrid = LinearGrid(M=Msgrid,N=Nsgrid,NFP=eq.NFP,sym=False) 

In [7]:
test = surface_current_field.compute(["b_t"],grid = sgrid)["b_t"]

In [8]:
test = surface_current_field.compute(["b_z"],grid = sgrid)["b_z"]

In [9]:
# Check parameters before iterations
surface_current_field.G

1.065272656253003

In [10]:
surface_current_field.I

7.074510494094354e-18

In [11]:
#surface_current_field.Phi_mn

In [12]:
surface_current_field.R_lmn

Array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  9.97526151e-01,  3.00000000e-01,  9.24016545e-02,
        0.00000000e+00,  1.38862261e-03,  0.00000000e+00, -2.67151718e-04,
        0.00000000e+00, -3.92485261e-07,  0.00000000e+00, -3.61965058e-06,
        0.00000000e+00], dtype=float64)

In [13]:
surface_current_field.Z_lmn

Array([ 3.23529818e-06,  0.00000000e+00, -1.18013927e-06,  0.00000000e+00,
        1.93635442e-04,  0.00000000e+00, -1.54306898e-03,  0.00000000e+00,
       -4.22559747e-02,  0.00000000e+00, -3.00000000e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],      dtype=float64)

In [14]:
#%pdb

In [15]:
# form the constraints for the problem, we only want the Phi_mn to vary so we will fix all other parameters of FourierCurrentPotentialField
constraints = (  # now fix G only
    FixParameters(
        surface_current_field,
        params={"G": True},
        #params={"I": False, "G": True, "R_lmn": False, "Z_lmn": False},
        #params={"I": True, "G": True, "R_lmn": True, "Z_lmn": True},
    ),
    FixParameters(eq,)
)

# Bn^2 minimization
obj_flux = QuadraticFlux(
    field=surface_current_field,  # the field to be optimized
    eq=eq,  # the equilibrium upon which the quadratic flux is being evaluated
    eval_grid=egrid,
    field_grid=sgrid,
    vacuum=True,
    # this last flag makes the returned cost at each grid point be
    # Bn * sqrt(|e_theta x e_zeta|), which then yields the
    # correct form of least squares problem to be
    # equivalent to the REGCOIl form
    #sqrt_area_weighting=True,
)

# Sigma regularization
obj_sigma = bVariation(
    surface_current_field=surface_current_field,
    source_grid=sgrid,
    weight = 1e1,#np.sqrt(data["lambda_regularization"][8]),
)  # we will set it to the sqrt of optimal weight from above,
# since DESC will square the weight when making the overall cost fxn

# Current regularization
obj_regularization = SurfaceCurrentRegularization(
    surface_current_field=surface_current_field,
    source_grid=sgrid,
    weight = 1e-13,#np.sqrt(data["lambda_regularization"][8]),
)  # we will set it to the sqrt of optimal weight from above,
# since DESC will square the weight when making the overall cost fxn

# Distance target
min_distance = 0.1
max_distance = 0.2

obj_dist = PlasmaVesselDistance(
    eq=eq,
    surface=surface_current_field,
    #surface_fixed=False,
    bounds=(min_distance,max_distance),
    plasma_grid=egrid,
    surface_grid=sgrid,
    use_softmin=True,
    alpha=1e3,
    weight=1e2
)

obj_distMod = PlasmaVesselDistanceMod(
    eq=eq,
    surface=surface_current_field,
    #surface_fixed=False,
    bounds=(min_distance,max_distance),
    plasma_grid=egrid,
    surface_grid=sgrid,
    use_softmin=True,
    alpha=1e3,
    weight=1e2
)

objs = (
    obj_dist,
#    obj_distMod,
)

objs += (
    obj_flux,
    #obj_sigma,
    #obj_regularization, # Technically a smooth sigma is all we need, so current regularization is redundant
)

objective = ObjectiveFunction(
    objs,
    #verbose=0,
    deriv_mode="looped"
)

#objective = ObjectiveFunction((obj_flux, obj_regularization, 
#                               obj_dist
#                              )
#                             )

In [16]:
optimizer = Optimizer("lsq-exact")

In [17]:
test1 = jnp.asarray([(jnp.sqrt(test.shape[0])).astype(int)])

In [18]:
test2 = jnp.asarray([((jnp.sqrt(test.shape[0])).astype(int))])

In [19]:
test2 = (jnp.sqrt(test.shape[0])).astype(int)

In [20]:
:

SyntaxError: invalid syntax (1305497275.py, line 1)

In [21]:
surface_current_field, result = optimizer.optimize(
    surface_current_field,
    objective,
    constraints,
    verbose=1,
    ftol=0,
    gtol=0,
    xtol=1e-16,
    # because it is a linear least-squares problem, it is exactly solvable in 1 iteration
    maxiter=1,
    # we must make sure that the trust region of the optimization algorithm is
    # large enough to allow the entire problem to be solved in one step, so set the
    # trust radius to be infinite
    options={
        "initial_trust_radius": np.inf,
        # we also will use the Cholesky factorization to solve
        # since the matrices involved are positive-definite
        "tr_method": "cho",
    },
)

Building objective: plasma-vessel distance
Precomputing transforms
Building objective: Quadratic flux
Precomputing transforms
Building objective: Fixed parameters
Building objective: Fixed parameters


AssertionError: 

In [ ]:
(surface_current_field, eq), result = optimizer.optimize(
    (surface_current_field, eq),
     objective,
    constraints,
    verbose=1,
    ftol=0,
    gtol=0,
    xtol=1e-16,
    # because it is a linear least-squares problem, it is exactly solvable in 1 iteration
    maxiter=1,
    # we must make sure that the trust region of the optimization algorithm is
    # large enough to allow the entire problem to be solved in one step, so set the
    # trust radius to be infinite
    options={
        "initial_trust_radius": np.inf,
        # we also will use the Cholesky factorization to solve
        # since the matrices involved are positive-definite
        "tr_method": "cho",
    },
)

In [ ]:
surface_current_field.G

In [ ]:
surface_current_field.I

In [ ]:
surface_current_field.Phi_mn

In [ ]:
surface_current_field.R_lmn

In [ ]:
surface_current_field.Z_lmn